## Project Overview
"""
This notebook implements a Naive/Basic RAG system that creates a Python learning assistant.
The system can answer questions about Python programming by retrieving relevant information
from a knowledge base and generating contextual responses.

What we'll build:
1. A document collection about Python programming concepts
2. A vector store for semantic search
3. A basic RAG pipeline for question answering
4. An interactive Q&A interface
"""


## Run this in your terminal before running the notebook:

pip install sentence-transformers
pip install faiss-cpu
pip install numpy
pip install scikit-learn
pip install google-generativeai
pip install python-dotenv


In [1]:
import numpy as np
import json
import re
import os
from typing import List, Dict, Tuple
from sentence_transformers import SentenceTransformer
import faiss
import google.generativeai as genai
from dotenv import load_dotenv

/home/mohdasimkhan/.pyenv/versions/3.10.2/envs/rags/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables
load_dotenv()

True

# Part 1: Knowledge Base Creation

In [3]:
# Part 1: Knowledge Base Creation
print("🔧 Setting up the Python Learning Knowledge Base...")

# Sample documents about Python programming
python_knowledge_base = [
    {
        "id": "py_001",
        "title": "Python Variables and Data Types",
        "content": """Python variables are created when you assign a value to them. Python has several built-in data types including integers (int), floating-point numbers (float), strings (str), booleans (bool), lists, tuples, dictionaries, and sets. Variables in Python are dynamically typed, meaning you don't need to declare their type explicitly. For example: name = 'Alice' creates a string variable, age = 25 creates an integer, and height = 5.6 creates a float."""
    },
    {
        "id": "py_002", 
        "title": "Python Functions",
        "content": """Functions in Python are defined using the 'def' keyword followed by the function name and parentheses. Functions can accept parameters and return values using the 'return' statement. They help organize code into reusable blocks. Example: def greet(name): return f'Hello, {name}!'. Functions can have default parameters, variable-length arguments (*args), and keyword arguments (**kwargs). Lambda functions are anonymous functions defined using the 'lambda' keyword for simple operations."""
    },
    {
        "id": "py_003",
        "title": "Python Lists and List Comprehensions", 
        "content": """Lists in Python are ordered, mutable collections that can contain elements of different data types. Lists are created using square brackets: my_list = [1, 2, 3, 'hello']. Common list methods include append(), extend(), insert(), remove(), pop(), and sort(). List comprehensions provide a concise way to create lists: [x**2 for x in range(10) if x % 2 == 0] creates a list of squares of even numbers from 0 to 8."""
    },
    {
        "id": "py_004",
        "title": "Python Dictionaries",
        "content": """Dictionaries in Python are unordered collections of key-value pairs. They are created using curly braces: person = {'name': 'John', 'age': 30}. Dictionaries are mutable and allow fast lookups by key. Common methods include keys(), values(), items(), get(), pop(), and update(). Dictionary comprehensions work similarly to list comprehensions: {x: x**2 for x in range(5)} creates a dictionary mapping numbers to their squares."""
    },
    {
        "id": "py_005",
        "title": "Python Loops and Iteration",
        "content": """Python supports two main types of loops: 'for' loops and 'while' loops. For loops iterate over sequences like lists, strings, or ranges: for i in range(5): print(i). While loops continue executing as long as a condition is True. Python also provides 'break' to exit loops early and 'continue' to skip to the next iteration. The enumerate() function provides both index and value when iterating: for index, value in enumerate(my_list)."""
    },
    {
        "id": "py_006",
        "title": "Python Classes and Objects",
        "content": """Python is an object-oriented programming language. Classes are blueprints for creating objects. They are defined using the 'class' keyword. Classes can have attributes (variables) and methods (functions). The __init__ method is the constructor that initializes new objects. Example: class Dog: def __init__(self, name): self.name = name. Objects are instances of classes created by calling the class: my_dog = Dog('Buddy'). Python supports inheritance, allowing classes to inherit attributes and methods from parent classes."""
    },
    {
        "id": "py_007",
        "title": "Python Exception Handling",
        "content": """Exception handling in Python uses try-except blocks to gracefully handle errors. The try block contains code that might raise an exception, while except blocks handle specific exceptions. You can catch specific exceptions like ValueError, TypeError, or use a general Exception. The finally block always executes regardless of whether an exception occurred. Example: try: result = 10/0 except ZeroDivisionError: print('Cannot divide by zero'). You can also raise custom exceptions using the 'raise' statement."""
    },
    {
        "id": "py_008",
        "title": "Python File Operations",
        "content": """Python provides built-in functions for file operations. The open() function is used to open files with different modes: 'r' for reading, 'w' for writing, 'a' for appending. Always close files using close() or use the 'with' statement for automatic file handling: with open('file.txt', 'r') as f: content = f.read(). Common file methods include read(), readline(), readlines(), write(), and writelines(). Python can handle text files and binary files, and supports various encodings like UTF-8."""
    },
    {
        "id": "py_009",
        "title": "Python Modules and Packages",
        "content": """Modules in Python are files containing Python code that can be imported and used in other programs. You can import modules using 'import module_name' or import specific functions using 'from module_name import function_name'. Packages are directories containing multiple modules with an __init__.py file. Python has a rich ecosystem of built-in modules like os, sys, math, datetime, and third-party packages available through pip. Virtual environments help manage package dependencies for different projects."""
    },
    {
        "id": "py_010",
        "title": "Python Decorators",
        "content": """Decorators in Python are a way to modify or extend the behavior of functions or classes without permanently modifying their code. They use the @ symbol and are placed above function definitions. Decorators are essentially functions that take another function as an argument and return a modified version. Common use cases include logging, timing, authentication, and caching. Example: @property decorator converts a method into a property, and @staticmethod creates methods that don't require instance or class references."""
    }
]


🔧 Setting up the Python Learning Knowledge Base...


# Part 2: RAG System Implementation

In [4]:
class NaiveRAG:
    def __init__(self, embedding_model_name: str = "all-MiniLM-L6-v2", gemini_api_key: str = None):
        """
        Initialize the Naive RAG system
        
        Args:
            embedding_model_name: Name of the sentence transformer model to use
            gemini_api_key: Google Gemini API key (or set GEMINI_API_KEY environment variable)
        """
        print(f"🔄 Initializing RAG system with model: {embedding_model_name}")
        
        # Initialize embedding model
        self.embedding_model = SentenceTransformer(embedding_model_name)
        
        # Initialize Gemini
        api_key = gemini_api_key or os.getenv('GEMINI_API_KEY')
        if not api_key:
            raise ValueError(
                "Gemini API key is required. Either pass it as gemini_api_key parameter "
                "or set GEMINI_API_KEY environment variable. "
                "Get your API key from: https://aistudio.google.com/app/apikey"
            )
        
        genai.configure(api_key=api_key)
        self.gemini_model = genai.GenerativeModel('gemini-1.5-flash')
        
        # Storage for documents and embeddings
        self.documents = []
        self.embeddings = None
        self.index = None
        
        print("✅ RAG system initialized successfully with Gemini")
    
    def add_documents(self, documents: List[Dict]):
        """
        Add documents to the knowledge base and create embeddings
        
        Args:
            documents: List of documents with 'id', 'title', and 'content' keys
        """
        print("🔄 Processing and embedding documents...")
        
        self.documents = documents
        
        # Create text chunks for embedding (title + content)
        texts = [f"{doc['title']} {doc['content']}" for doc in documents]
        
        # Generate embeddings
        self.embeddings = self.embedding_model.encode(texts)
        
        # Create FAISS index for fast similarity search
        dimension = self.embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dimension)  # Inner product for similarity
        
        # Normalize embeddings for cosine similarity
        faiss.normalize_L2(self.embeddings)
        self.index.add(self.embeddings)
        
        print(f"✅ {len(documents)} documents processed and indexed")
    
    def retrieve_documents(self, query: str, top_k: int = 3) -> List[Dict]:
        """
        Retrieve most relevant documents for a given query
        
        Args:
            query: User query string
            top_k: Number of top documents to retrieve
            
        Returns:
            List of relevant documents with similarity scores
        """
        # Encode query
        query_embedding = self.embedding_model.encode([query])
        faiss.normalize_L2(query_embedding)
        
        # Search for similar documents
        scores, indices = self.index.search(query_embedding, top_k)
        
        # Prepare results
        results = []
        for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
            results.append({
                'document': self.documents[idx],
                'similarity_score': float(score),
                'rank': i + 1
            })
        
        return results
    
    def generate_answer(self, query: str, retrieved_docs: List[Dict]) -> str:
        """
        Generate an answer using the query and retrieved documents with Gemini
        
        Args:
            query: User query
            retrieved_docs: List of retrieved documents
            
        Returns:
            Generated answer string
        """
        # Prepare context from retrieved documents
        context_parts = []
        for doc_info in retrieved_docs:
            doc = doc_info['document']
            context_parts.append(f"Title: {doc['title']}\nContent: {doc['content']}")
        
        context = "\n\n".join(context_parts)
        
        # Create prompt for generation
        prompt = f"""You are a helpful Python programming assistant. Use the provided context to answer the user's question accurately and comprehensively.

        Context Information:
        {context}
        
        User Question: {query}
        
        Instructions:
        - Provide a clear, helpful answer based on the context provided
        - If the context contains relevant information, use it to give a detailed response
        - If the context doesn't contain enough information, clearly state what you know and what might be missing
        - Include practical examples when appropriate
        - Keep the answer focused on Python programming
        
        Answer:"""

        # Generate answer using Gemini
        try:
            response = self.gemini_model.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    candidate_count=1,
                    max_output_tokens=500,
                    temperature=0.3,
                )
            )
            return response.text
        except Exception as e:
            return f"Sorry, I encountered an error generating the response: {str(e)}"
    
    def ask(self, query: str, top_k: int = 3, verbose: bool = True) -> Dict:
        """
        Complete RAG pipeline: retrieve documents and generate answer
        
        Args:
            query: User question
            top_k: Number of documents to retrieve
            verbose: Whether to print intermediate steps
            
        Returns:
            Dictionary containing the answer and metadata
        """
        if verbose:
            print(f"🔍 Processing query: '{query}'")
        
        # Step 1: Retrieve relevant documents
        retrieved_docs = self.retrieve_documents(query, top_k)
        
        if verbose:
            print(f"📚 Retrieved {len(retrieved_docs)} relevant documents:")
            for doc_info in retrieved_docs:
                doc = doc_info['document']
                score = doc_info['similarity_score']
                print(f"  - {doc['title']} (similarity: {score:.3f})")
        
        # Step 2: Generate answer
        answer = self.generate_answer(query, retrieved_docs)
        
        if verbose:
            print(f"💡 Generated answer:")
            print(f"   {answer}")
        
        return {
            'query': query,
            'answer': answer,
            'retrieved_documents': retrieved_docs,
            'num_retrieved': len(retrieved_docs)
        }

# Part 3: Initialize and Setup the RAG System


In [5]:
print("\n" + "="*50)
print("🚀 INITIALIZING PYTHON LEARNING ASSISTANT WITH GEMINI")
print("="*50)




🚀 INITIALIZING PYTHON LEARNING ASSISTANT WITH GEMINI


# Setup Gemini API Key
"""
To use this system, you need a Gemini API key:
1. Go to https://aistudio.google.com/app/apikey
2. Create a new API key
3. Either:
   - Set it as environment variable: export GEMINI_API_KEY="your-api-key"
   - Create a .env file with: GEMINI_API_KEY=your-api-key
   - Pass it directly to the NaiveRAG constructor

For this demo, uncomment and set your API key below:
"""

# Option 1: Set your API key directly (not recommended for production)
# GEMINI_API_KEY = "your-api-key-here"
# os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY


In [6]:
# Option 2: Use environment variable or .env file (recommended)
if not os.getenv('GEMINI_API_KEY'):
    print("⚠️  GEMINI_API_KEY not found in environment variables.")
    print("   Please set your API key using one of these methods:")
    print("   1. Set environment variable: export GEMINI_API_KEY='your-key'")
    print("   2. Create .env file with: GEMINI_API_KEY=your-key")
    print("   3. Uncomment and set the API key in the code above")
    print("   Get your API key from: https://aistudio.google.com/app/apikey")
    
    # For demonstration, we'll create a mock instance
    print("\n   Creating a mock instance for demonstration...")
    print("   (Replace with actual API key to test the system)")
    
    # Uncomment the next two lines and add your API key to test
    # api_key = "your-actual-api-key-here"
    # rag_system = NaiveRAG(gemini_api_key=api_key)
    
    # For now, create a placeholder
    rag_system = None
else:
    # Create RAG instance with Gemini
    rag_system = NaiveRAG()

🔄 Initializing RAG system with model: all-MiniLM-L6-v2
✅ RAG system initialized successfully with Gemini


# Add documents to the system (only if we have a valid instance)

In [7]:
if rag_system is not None:
    rag_system.add_documents(python_knowledge_base)
else:
    print("   Skipping document indexing - please set up API key first")

🔄 Processing and embedding documents...
✅ 10 documents processed and indexed


In [8]:
print("\n" + "="*50)
print("🧪 TESTING THE RAG SYSTEM WITH GEMINI")
print("="*50)


🧪 TESTING THE RAG SYSTEM WITH GEMINI


# Part 4: Interactive Testing and Demonstration

In [15]:
print("\n" + "="*50)
print("🧪 TESTING THE RAG SYSTEM WITH GEMINI")
print("="*50)

# Test queries with categories for better organization
test_queries = [
    ("Basic Syntax", "How do I create a function in Python?"),
    ("Data Types", "What are the different data types in Python?"),
    ("Advanced Features", "How do list comprehensions work?"),
    ("Error Handling", "How can I handle errors in Python?"),
    ("Data Structures", "What is the difference between lists and dictionaries?")
]

def print_separator(title="", char="=", length=60):
    """Print a formatted separator line"""
    if title:
        padding = (length - len(title) - 2) // 2
        print(char * padding + f" {title} " + char * padding)
    else:
        print(char * length)

def print_query_header(query_num, category, question):
    """Print a formatted query header"""
    print("\n" + "🔹" * 20)
    print(f"📋 **TEST QUERY #{query_num}**")
    print(f"📂 **Category:** {category}")
    print(f"❓ **Question:** {question}")
    print("🔹" * 20)

def print_result_section(title, content=""):
    """Print a formatted result section"""
    print(f"\n🎯 **{title.upper()}:**")
    if content:
        print(f"   {content}")

if rag_system is not None:
    print("\n🔍 **RUNNING COMPREHENSIVE TEST SUITE...**\n")
    print_separator("GEMINI-POWERED RAG EVALUATION", "=", 60)

    for i, (category, query) in enumerate(test_queries, 1):
        try:
            # Print formatted query header
            print_query_header(i, category, query)
            
            # Process the query
            print("\n🤔 **Processing with Gemini...**")
            result = rag_system.ask(query, top_k=2, verbose=False)
            
            # Print retrieved documents
            print_result_section("RETRIEVED DOCUMENTS")
            for j, doc_info in enumerate(result['retrieved_documents'], 1):
                doc = doc_info['document']
                score = doc_info['similarity_score']
                print(f"   {j}. 📄 **{doc['title']}** (Relevance: {score:.3f})")
            
            # Print generated answer
            print_result_section("GEMINI RESPONSE")
            # Split long answers into paragraphs for better readability
            answer_lines = result['answer'].split('\n')
            for line in answer_lines:
                if line.strip():
                    print(f"   {line.strip()}")
            
            # Print performance metrics
            print_result_section("PERFORMANCE METRICS")
            print(f"   • Documents Retrieved: {result['num_retrieved']}")
            print(f"   • Query Category: {category}")
            print(f"   • Status: ✅ SUCCESS")
            
            print("\n" + "🔸" * 30 + " END OF QUERY " + "🔸" * 30)
            
        except Exception as e:
            print_result_section("ERROR")
            print(f"   ❌ **Error:** {str(e)}")
            print("   💡 **Suggestion:** Make sure your Gemini API key is valid and you have credits")
            print(f"   📊 **Status:** ❌ FAILED")
            print("\n" + "🔸" * 30 + " END OF QUERY " + "🔸" * 30)
    
    print("\n" + "="*60)
    print("🎉 **TEST SUITE COMPLETED**")
    print("="*60)
    
else:
    print("\n⚠️  **CANNOT RUN TESTS WITHOUT A VALID GEMINI API KEY**")
    print("="*60)
    print("📋 **SETUP INSTRUCTIONS:**")
    print("   1. Get API key from: https://aistudio.google.com/app/apikey")
    print("   2. Set environment variable: export GEMINI_API_KEY='your-key'")
    print("   3. Or create .env file with: GEMINI_API_KEY=your-key")
    print("   4. Restart the system")
    print("="*60)


🧪 TESTING THE RAG SYSTEM WITH GEMINI

🔍 **RUNNING COMPREHENSIVE TEST SUITE...**

============== GEMINI-POWERED RAG EVALUATION ==============

🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹
📋 **TEST QUERY #1**
📂 **Category:** Basic Syntax
❓ **Question:** How do I create a function in Python?
🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹

🤔 **Processing with Gemini...**

🎯 **RETRIEVED DOCUMENTS:**
   1. 📄 **Python Functions** (Relevance: 0.632)
   2. 📄 **Python Lists and List Comprehensions** (Relevance: 0.486)

🎯 **GEMINI RESPONSE:**
   In Python, you create a function using the `def` keyword, followed by the function name, parentheses `()`, and a colon `:`.  The function body, indented below the `def` line, contains the code to be executed.
   The provided context explains this well:  "Functions in Python are defined using the 'def' keyword followed by the function name and parentheses. Functions can accept parameters and return values using the 'return' statement. They help organize code into reusable blocks."
   Here's a breakdown w

# Part 5: Interactive Q&A Function

In [10]:
def interactive_qa():
    """
    Interactive question-answering session with the RAG system
    """
    if rag_system is None:
        print("❌ Cannot start interactive mode without a valid Gemini API key")
        print("   Please set up your API key and restart the system")
        return
    
    print("\n" + "="*50)
    print("🤖 PYTHON LEARNING ASSISTANT - INTERACTIVE MODE (Powered by Gemini)")
    print("="*50)
    print("Ask me anything about Python programming!")
    print("Type 'quit' to exit, 'help' for examples")
    print("-"*50)
    
    example_questions = [
        "How do I create a list in Python?",
        "What is a Python decorator?", 
        "How do I read a file in Python?",
        "What are Python classes?",
        "How do loops work in Python?"
    ]
    
    while True:
        try:
            user_query = input("\n❓ Your question: ").strip()
            
            if user_query.lower() in ['quit', 'exit', 'q']:
                print("👋 Goodbye! Happy learning!")
                break
            elif user_query.lower() == 'help':
                print("\n💡 Example questions you can ask:")
                for example in example_questions:
                    print(f"   • {example}")
                continue
            elif not user_query:
                print("Please enter a question or type 'help' for examples.")
                continue
            
            # Process the query
            print("\n🤔 Thinking with Gemini...")
            result = rag_system.ask(user_query, top_k=2, verbose=False)
            
            print(f"\n🤖 Answer:")
            print(f"   {result['answer']}")
            
            # Show sources
            print(f"\n📚 Sources consulted:")
            for doc_info in result['retrieved_documents']:
                doc = doc_info['document']
                score = doc_info['similarity_score']
                print(f"   • {doc['title']} (relevance: {score:.2f})")
                
        except KeyboardInterrupt:
            print("\n\n👋 Goodbye! Happy learning!")
            break
        except Exception as e:
            print(f"\n❌ An error occurred: {str(e)}")
            if "quota" in str(e).lower() or "billing" in str(e).lower():
                print("   This might be a Gemini API quota or billing issue.")
                print("   Check your API usage at: https://aistudio.google.com/")


# Part 6: Evaluation and Analysis Functions

In [11]:
def evaluate_retrieval_quality():
    """
    Evaluate the quality of document retrieval
    """
    if rag_system is None:
        print("❌ Cannot evaluate without a valid RAG system")
        return
        
    print("\n" + "="*50)
    print("📊 RETRIEVAL QUALITY EVALUATION")
    print("="*50)
    
    evaluation_queries = [
        ("What are Python variables?", ["py_001"]),  # Should retrieve variables doc
        ("How do I define a function?", ["py_002"]),  # Should retrieve functions doc
        ("Tell me about loops", ["py_005"]),  # Should retrieve loops doc
        ("How do I handle exceptions?", ["py_007"]),  # Should retrieve exceptions doc
    ]
    
    total_score = 0
    for query, expected_docs in evaluation_queries:
        print(f"\n🔍 Query: {query}")
        retrieved = rag_system.retrieve_documents(query, top_k=3)
        
        retrieved_ids = [doc['document']['id'] for doc in retrieved]
        print(f"   Expected: {expected_docs}")
        print(f"   Retrieved: {retrieved_ids[:len(expected_docs)]}")
        
        # Check if expected document is in top results
        score = 1 if any(doc_id in retrieved_ids[:len(expected_docs)] for doc_id in expected_docs) else 0
        total_score += score
        print(f"   ✅ Correct: {score}")
    
    accuracy = total_score / len(evaluation_queries)
    print(f"\n📈 Overall Retrieval Accuracy: {accuracy:.2f} ({total_score}/{len(evaluation_queries)})")


In [12]:
def analyze_embeddings():
    """
    Analyze the embedding space and document similarities
    """
    if rag_system is None:
        print("❌ Cannot analyze embeddings without a valid RAG system")
        return
        
    print("\n" + "="*50)
    print("🔬 EMBEDDING SPACE ANALYSIS")  
    print("="*50)
    
    # Calculate pairwise similarities between all documents
    similarities = np.dot(rag_system.embeddings, rag_system.embeddings.T)
    
    # Find most similar document pairs
    print("\n🔗 Most similar document pairs:")
    for i in range(len(rag_system.documents)):
        for j in range(i+1, len(rag_system.documents)):
            similarity = similarities[i][j]
            if similarity > 0.3:  # Threshold for "similar"
                doc1 = rag_system.documents[i]
                doc2 = rag_system.documents[j]
                print(f"   • '{doc1['title']}' ↔ '{doc2['title']}' (similarity: {similarity:.3f})")

# Part 7: Run Demonstrations

In [13]:
if __name__ == "__main__":
    # Run evaluation
    evaluate_retrieval_quality()
    
    # Analyze embeddings
    analyze_embeddings()
    
    # Uncomment the line below to start interactive mode
    # interactive_qa()
    
    print("\n" + "="*50)
    print("✨ NAIVE RAG IMPLEMENTATION WITH GEMINI COMPLETE!")
    print("="*50)
    print("🎯 What we built:")
    print("   • Document embedding and indexing system")
    print("   • Semantic similarity search using FAISS")
    print("   • Text generation using Google Gemini")
    print("   • Complete RAG pipeline with evaluation")
    print("\n💡 To start interactive mode, call: interactive_qa()")
    print("🔧 To experiment further, try:")
    print("   • Adding more documents to the knowledge base")
    print("   • Adjusting the top_k parameter for retrieval")
    print("   • Using different embedding models")
    print("   • Implementing query expansion techniques")
    print("   • Experimenting with different Gemini models (gemini-1.5-pro)")



📊 RETRIEVAL QUALITY EVALUATION

🔍 Query: What are Python variables?
   Expected: ['py_001']
   Retrieved: ['py_001']
   ✅ Correct: 1

🔍 Query: How do I define a function?
   Expected: ['py_002']
   Retrieved: ['py_002']
   ✅ Correct: 1

🔍 Query: Tell me about loops
   Expected: ['py_005']
   Retrieved: ['py_005']
   ✅ Correct: 1

🔍 Query: How do I handle exceptions?
   Expected: ['py_007']
   Retrieved: ['py_007']
   ✅ Correct: 1

📈 Overall Retrieval Accuracy: 1.00 (4/4)

🔬 EMBEDDING SPACE ANALYSIS

🔗 Most similar document pairs:
   • 'Python Variables and Data Types' ↔ 'Python Functions' (similarity: 0.488)
   • 'Python Variables and Data Types' ↔ 'Python Lists and List Comprehensions' (similarity: 0.524)
   • 'Python Variables and Data Types' ↔ 'Python Dictionaries' (similarity: 0.522)
   • 'Python Variables and Data Types' ↔ 'Python Loops and Iteration' (similarity: 0.419)
   • 'Python Variables and Data Types' ↔ 'Python Classes and Objects' (similarity: 0.536)
   • 'Python Variabl